# Imports

In [86]:
import csv
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn import metrics
import math
from tkinter import *

import sys
sys.path.append('interface')

# Allows jupyter notebook to be imported
import jupyter_import

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

# User Interface

In [87]:
from interface.main import Main

# Preprocessing

# Imports

In [88]:
import csv
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn import metrics
import math

# sys.path.append('interface')

# Allows jupyter notebook to be imported
import jupyter_import

# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

# User Interface

In [89]:
from interface import *
from interface.home import Home

# Preprocessing

In [90]:
from data_preproc.Preprocess import preprocess, Normalize

# Feature Selection

In [91]:
from data_preproc.CFS import cfs_algo
from data_preproc.RFE import rfe_algo
from data_preproc.RR import ridge_algo

# Algorithms

## Base Predictors

In [92]:
from pred_mdls.base.Complement_Naive_Bayes import complement_naive_bayes_model
from pred_mdls.base.Decision_Tree import decision_tree_model
from pred_mdls.base.Logistic_Regression import logistic_regression_model
from pred_mdls.base.Multi_Layer_Perceptron import multi_layer_perceptron_model
from pred_mdls.base.Naive_Bayes import naive_bayes_model

## Ensemble Predictors

In [93]:
from pred_mdls.ensemble.Random_Forest import random_forest_model
from pred_mdls.ensemble.Rotation_Forest import rotation_forest_model
from pred_mdls.ensemble.Voting import voting_model

# Evaluation Metrics

In [94]:
from pf_eval.AUC_ROC import auc_roc_model
from pf_eval.F1_Score import f1_model
from pf_eval.CSV import write_results
from pf_eval.Confusion_Matrix import confusion_matrix_model

# Additional Functions

In [95]:
def data_conversion(data):
    for i in range(len(data)):
        if data[i] == b'N' or data[i] == b'false' or data[i] == b'no':
            data[i] = 0
        else:
            data[i] = 1
    return data

def read_data(filename):
    data = arff.loadarff(filename)
    loaddata = pd.DataFrame(data[0])
    return loaddata

def process_data(loaddata,features):
    # Features are selected based on CFS
    software_metrics = np.array(loaddata[features])
    labels = np.array(loaddata['Defective'])
    return software_metrics,labels

def train_data(software_metrics,labels):
    X_train, X_test, y_train, y_test = train_test_split(software_metrics, labels, test_size = 0.1)
    y_train = y_train.astype('str')
    y_test = y_test.astype('str')
    return X_train, X_test, y_train, y_test

def evaluate_data(model,model_name,X_test,y_test):
    auc_score = auc_roc_model(model,X_test,y_test)
    f1_score = f1_model(model,X_test,y_test)
    fpr,fnr = confusion_matrix_model(model,X_test,y_test)
    return auc_score,f1_score,fpr,fnr

def translate(result):
    count = 1
    res = []
    while count <= 3:
        for i in range(len(result[0])):
            res.append([result[0][i], result[1][((i+1)*count)-1],result[2][((i+1)*count)-1]])
        count += 1
    return res

# Result writers

In [96]:
def main_writer(header,result):
    #Writes the output of a single dataset for main function
    filters = ['No filter','CFS','RFE']
    with open('pred_results.csv','w',encoding='UTF8', newline='') as file:
        res = csv.writer(file)
        for i in range(len(filters)):
            res.writerow('')
            res.writerow([filters[i]])
            res.writerow(header)
            res.writerow([result[0][0]] + result[0][1][i*8:i*8+8])
            res.writerow([result[1][0]] + result[1][1][i*8:i*8+8])
    
def run(datasets, savename, repository):
    #Writes the output of multiple datasets for the main function
    header = ['Model name','Complement Naive Bayes','Decision Tree','Logistic regression',
                        'Multi Layer Perceptron','Naive Bayes','Random Forest','Rotation Forest','Voting']
    filters = ['(All)','(CFS)','(RFE)']
    with open(savename,'w',encoding='UTF8', newline='') as file:
        results = []
        for ds in datasets:
            if repository == 'NASA':
                results.append(main('datasets/NASA/' + ds + '.txt')[0])
            else:
                results.append(main('datasets/PROMISE/' + ds + '.txt')[0])
        res = csv.writer(file)
        for k in range(len(results[0])):
            res.writerow([results[0][k][0]])
            res.writerow(header)
            for j in range(len(results)):
                for i in range(len(filters)):
                    res.writerow([datasets[j]+filters[i]] + results[j][k][1][i*8:i*8+8])
            if k != len(results[0])-1:
                res.writerow('')
        #res.writerow([results[0][0][0]])
        #res.writerow(header)        
        #for j in range(len(results)):
            #for i in range(len(filters)):
                #res.writerow([datasets[j]+filters[i]] + results[j][0][1][i*8:i*8+8])
        #res.writerow('')
        #res.writerow([results[0][1][0]])
        #res.writerow(header)
        #for j in range(len(results)):
            #for i in range(len(filters)):
                #res.writerow([datasets[j]+filters[i]] + results[j][1][1][i*8:i*8+8])
        #res.writerow('')
        #res.writerow([results[0][2][0]])
        #res.writerow(header)
        #for j in range(len(results)):
            #for i in range(len(filters)):
                #res.writerow([datasets[j]+filters[i]] + results[j][2][1][i*8:i*8+8])
        #res.writerow('')
        #res.writerow([results[0][3][0]])
        #res.writerow(header)
        #for j in range(len(results)):
            #for i in range(len(filters)):
                #res.writerow([datasets[j]+filters[i]] + results[j][3][1][i*8:i*8+8])

# Main

In [98]:
def main_run(filename):
    # Read the file
    loaddata = read_data(filename)
    loaddata = Normalize(loaddata)
    SM = np.array(loaddata.iloc[:,:-1]) #Software metrics
    L = data_conversion(np.array(loaddata.iloc[:,-1])).astype(int) #Labels
    data = [SM,L]
    model_name = ['Complement Naive Bayes','Decision Tree','Logistic regression',
                    'Multi Layer Perceptron','Naive Bayes','Random Forest','Rotation Forest','Voting'] #Models used
    
    # ===== Feature Selection ====== #

    # ==== CFS ==== #
    cfs, cfs_selections = cfs_algo(data,10)
    # ============= #

    # ===== RFE ======== #
    rfe, rfe_selections = rfe_algo(data,10)
    # ================== #
    
    # ========= Preprocessing ============= #
    pp = preprocess(loaddata)
    pp_cfs = preprocess(loaddata, cfs_selections)
    pp_rfe = preprocess(loaddata, rfe_selections)

    pp_arr = [pp,pp_cfs,pp_rfe]
    pp_name = ['No filters','CFS Feature Selection','RFE Feature Selection']
    # ====================================== #

    result = []
    arr_size = len(model_name)*len(pp_name) #Result array size
    auc_arr = [0]*arr_size
    f1_arr = [0]*arr_size
    fpr_arr = [0]*arr_size
    fnr_arr = [0]*arr_size
    header = []
    folds = 5
    for j,pp in enumerate(pp_arr):
        for i in range(folds):
            data = [pp[i][0],pp[i][2]]

            # ======== Model Creation =========== #
            # Base Predictors
            cnb = complement_naive_bayes_model(data)
            dt = decision_tree_model(data)
            lr = logistic_regression_model(data)
            mlp = multi_layer_perceptron_model(data)
            nb = naive_bayes_model(data)

            args = [1000]
            # Ensemble Predictors
            rf = random_forest_model(data,args)
            rof = rotation_forest_model(data,args)
            vt = voting_model(data,args)
            # ==================================== #

            models = [cnb,dt,lr,mlp,nb,rf,rof,vt] 
            for k in range(len(models)):
                auc_score,f1_score,fpr,fnr = evaluate_data(models[k],model_name[k],pp[i][1],pp[i][3])
                if math.isnan(auc_score):
                    print(model_name[k], auc_score)
                    auc_score = 0
                auc_arr[(j*8)+k] += auc_score
                f1_arr[(j*8)+k] += f1_score
                fpr_arr[(j*8)+k] += fpr
                fnr_arr[(j*8)+k] += fnr
    for i in range(len(models)*3):
        auc_arr[i] /= folds
        auc_arr[i] = round(auc_arr[i],3)
        f1_arr[i] /= folds
        f1_arr[i] = round(f1_arr[i],3)
        fpr_arr[i] /= folds
        fpr_arr[i] = round(fpr_arr[i],3)
        fnr_arr[i] /= folds
        fnr_arr[i] = round(fnr_arr[i],3)
    header.append('Model Name')
    header += model_name
    result.append(('AUC', auc_arr))
    result.append(('F1 Score', f1_arr))
    result.append(('False Positive Rate', fpr_arr))
    result.append(('False Negative Rate', fnr_arr))
    #Print filename upon completion
    print(filename)
    return result, header     
            
if __name__=='__main__':
    # N_filenames = ['CM1.arff','JM1.arff','KC1.arff','KC3.arff',
    #                'KC4.arff','MC1.arff','MC2.arff','MW1.arff',
    #                'PC1.arff','PC2.arff','PC3.arff','PC4.arff','PC5.arff']
    # P_filenames = ['cm1.arff','jm1.arff','kc1.arff','kc2.arff','pc1.arff']
    # run(N_filenames,'NASA.csv','NASA')
    # run(P_filenames,'PROMISE.csv','PROMISE')
    root = Tk()
    main_root = Main(root)
    print(main_root.get_results())
    root.title('Prediction software')
    root.mainloop()

0 0
<bound method Main.get_results of <interface.main.Main object at 0x0000022138B575E0>>
